In [2]:
import carla
import random

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [5]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
my_map = world.get_map()
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

Own code starts here.

In [6]:
my_car_bp = world.get_blueprint_library().find('vehicle.tesla.model3')

In [8]:
ego = world.spawn_actor(blueprint=my_car_bp, transform=random.choice(spawn_points))
ego.set_autopilot(True)